# DEMO PROGRAM SKRIPSI
AMIRAH DZATUL HIMMAH <BR>
2002871

In [ ]:
!pip install keras-cv

In [ ]:
# insert dataset
from google.colab import drive
drive.mount('/content/drive')

!unzip '/content/drive/MyDrive/skripsi/dataset_split.zip'

In [ ]:
# Dictitonary for experiment configurations
EXPERIMENT_CONFIGS = {
    1: {"name": "experiment_1_VanillaCnnN", "model": "VanillaCNN", "augment": "No", "trainable": False},
    2: {"name": "experiment_2_VanillaCnnS", "model": "VanillaCNN", "augment": "Simple", "trainable": False},
    3: {"name": "experiment_3_VanillaCnnR", "model": "VanillaCNN", "augment": "Randaugment", "trainable": False},
    4: {"name": "experiment_4_InceptionV3NF", "model": "InceptionV3", "augment": "No", "trainable": False},
    5: {"name": "experiment_5_InceptionV3NU", "model": "InceptionV3", "augment": "No", "trainable": True},
    6: {"name": "experiment_6_InceptionV3SF", "model": "InceptionV3", "augment": "Simple", "trainable": False},
    7: {"name": "experiment_7_InceptionV3SU", "model": "InceptionV3", "augment": "Simple", "trainable": True},
    8: {"name": "experiment_8_InceptionV3RF", "model": "InceptionV3", "augment": "Randaugment", "trainable": False},
    9: {"name": "experiment_9_InceptionV3RU", "model": "InceptionV3", "augment": "Randaugment", "trainable": True},
    10: {"name": "experiment_10_ResNet50NF", "model": "ResNet50", "augment": "No", "trainable": False},
    11: {"name": "experiment_11_ResNet50NU", "model": "ResNet50", "augment": "No", "trainable": True},
    12: {"name": "experiment_12_ResNet50SF", "model": "ResNet50", "augment": "Simple", "trainable": False},
    13: {"name": "experiment_13_ResNet50SU", "model": "ResNet50", "augment": "Simple", "trainable": True},
    14: {"name": "experiment_14_ResNet50RF", "model": "ResNet50", "augment": "Randaugment", "trainable": False},
    15: {"name": "experiment_15_ResNet50RU", "model": "ResNet50", "augment": "Randaugment", "trainable": True},
}

# Select experiment
selected_experiment = 14  # Change this number to select experiment

# Set experiment configuration
config = EXPERIMENT_CONFIGS[selected_experiment]
EXPERIMENT_NAME = config["name"]
MODEL_NAME = config["model"]
AUGMENTATION_TYPE = config["augment"]
BASE_MODEL_TRAINABLE = config["trainable"]

print(f"Running {EXPERIMENT_NAME} with:")
print(f" - Model: {MODEL_NAME}")
print(f" - Augmentation: {AUGMENTATION_TYPE}")
print(f" - Base Model Trainable: {BASE_MODEL_TRAINABLE}")

In [ ]:
import os
import pathlib
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from keras import layers
from keras import callbacks
from keras_cv.layers import RandAugment
from keras.optimizers import Adam
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
# Configuration
DRIVE_DIR = '/content/drive/MyDrive/skripsi/eksperimen' # Change this to your own directory
BATCH_SIZE = 16
EPOCHS = 100

# Adjust the image size based on the model
def get_image_size(model_name):
    if model_name == "VanillaCNN":
        return 256  # Vanilla CNN needs 256x256
    elif model_name == "InceptionV3":
        return 299  # InceptionV3 needs 299x299
    elif model_name == "ResNet50":
        return 224  # ResNet50 needs 224x224
    else:
        raise ValueError(f"Model {model_name} unrecognized")

IMAGE_SIZE = get_image_size(MODEL_NAME)

TRAIN_DIR = 'dataset_split/train'
VAL_DIR = 'dataset_split/val'
TEST_DIR = 'dataset_split/test'

# Paths
EXPERIMENT_DIR = os.path.join(DRIVE_DIR, EXPERIMENT_NAME)
os.makedirs(EXPERIMENT_DIR, exist_ok=True)
HISTORY_PATH = os.path.join(EXPERIMENT_DIR, 'training_history.csv')
BEST_MODEL_PATH = os.path.join(EXPERIMENT_DIR, 'best_model.keras')
EPOCH_MODEL_PATH = os.path.join(EXPERIMENT_DIR, 'checkpoint.keras')
PLOT_PATH = os.path.join(EXPERIMENT_DIR, 'plot.png')

In [ ]:
# Dataset Loader
def load_dataset(data_dir, image_size):
    data_dir = pathlib.Path(data_dir)
    class_names = sorted([item.name for item in data_dir.glob('*') if item.is_dir()])
    class_dict = {name: idx for idx, name in enumerate(class_names)}
    images, labels = [], []

    for class_name in class_names:
        for image_path in (data_dir / class_name).glob('*'):
            img = tf.keras.utils.load_img(image_path, target_size=image_size)
            images.append(tf.keras.utils.img_to_array(img))
            labels.append(class_dict[class_name])

    return np.array(images), np.array(labels), class_names

# Dataset Preparation
def prepare_dataset(x, y, augment_type):
    dataset = tf.data.Dataset.from_tensor_slices((x, y))
    dataset = dataset.shuffle(len(x)).batch(BATCH_SIZE)

    if augment_type == "Simple":
        simple_aug = keras.Sequential(
            [
                layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
                layers.RandomFlip("horizontal"),
                layers.RandomRotation(factor=0.1),
                layers.RandomBrightness(factor=0.2),
            ]
        )
        dataset = dataset.map(lambda x, y: (simple_aug(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    elif augment_type == "Randaugment":
        rand_augment = RandAugment(value_range=(0, 255), augmentations_per_image=3, magnitude=0.5)
        dataset = dataset.map(lambda x, y: (rand_augment(x), y), num_parallel_calls=tf.data.AUTOTUNE)

    return dataset.prefetch(tf.data.AUTOTUNE)

# Model Vanilla CNN
def build_vanilla_cnn_model(num_classes):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3)),

        # 1st Conv Layer + Max Pooling
        layers.Conv2D(32, (3, 3), activation='relu'),  # 32 filters, kernel size 3x3
        layers.MaxPooling2D(2, 2),  # Max Pooling with 2x2 pool size

        # 2nd Conv Layer + Max Pooling
        layers.Conv2D(32, (3, 3), activation='relu'),  # 32 filters, kernel size 3x3
        layers.MaxPooling2D(2, 2),  # Max Pooling with 2x2 pool size

        # 3rd Conv Layer + Max Pooling
        layers.Conv2D(64, (3, 3), activation='relu'),  # 64 filters, kernel size 3x3
        layers.MaxPooling2D(2, 2),  # Max Pooling with 2x2 pool size

        # 4th Conv Layer + Max Pooling
        layers.Conv2D(64, (3, 3), activation='relu'),  # 64 filters, kernel size 3x3
        layers.MaxPooling2D(2, 2),  # Max Pooling with 2x2 pool size

        # Flatten the output of the last Conv layer to feed into the Dense layers
        layers.Flatten(),

        # First Dense Layer
        layers.Dense(128, activation='relu'),  # Fully connected layer with 128 units

        # Second Dense Layer (Output Layer for 5 categories)
        layers.Dense(num_classes, activation='softmax')  # Output layer with 5 units for 5 categories
    ])
    return model

# Model Definition
def build_model(model_name, num_classes, trainable):
    image_size = get_image_size(model_name)  # Get image size based on model

    if model_name == "VanillaCNN":
        # if model_name is VanillaCNN, build the model using the function
        return build_vanilla_cnn_model(num_classes)

    elif model_name == "ResNet50":
        base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    elif model_name == "InceptionV3":
        base_model = tf.keras.applications.InceptionV3(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))

    else:
        # if model_name is not recognized, raise an error
        raise ValueError(f"Model {model_name} not supported in this configuration.")

    # Set base model to trainable or not
    base_model.trainable = trainable

    # Input layer
    inputs = layers.Input(shape=(image_size, image_size, 3))

    # Preprocessing based on model
    if model_name == "ResNet50":
        x = tf.keras.applications.resnet50.preprocess_input(inputs)
    elif model_name == "InceptionV3":
        x = tf.keras.applications.inception_v3.preprocess_input(inputs)

    # Process the input through the base model
    x = base_model(x)
    x = layers.GlobalAveragePooling2D()(x)  # Global Average Pooling
    outputs = layers.Dense(num_classes, activation='softmax')(x)  # Classification layer

    # Kembalikan model
    return tf.keras.Model(inputs, outputs)


In [ ]:
# Load datasets
(x_train, y_train, _), (x_val, y_val, _), (x_test, y_test, _) = (
    load_dataset(TRAIN_DIR, (IMAGE_SIZE, IMAGE_SIZE)),
    load_dataset(VAL_DIR, (IMAGE_SIZE, IMAGE_SIZE)),
    load_dataset(TEST_DIR, (IMAGE_SIZE, IMAGE_SIZE))
)

train_ds = prepare_dataset(x_train, y_train, AUGMENTATION_TYPE)
val_ds = prepare_dataset(x_val, y_val, "No")
test_ds = prepare_dataset(x_test, y_test, "No")

print(f"Train dataset size: {len(x_train)}")
print(f"Validation dataset size: {len(x_val)}")
print(f"Test dataset size: {len(x_test)}")

Train dataset size: 335
Validation dataset size: 82
Test dataset size: 102


In [ ]:
# Resume Training or Initialize Model
initial_epoch = 0
latest_epoch_model = None

# Check for the latest epoch checkpoint
if os.path.exists(EXPERIMENT_DIR):
    if os.path.exists(EPOCH_MODEL_PATH):
        history_df = pd.read_csv(HISTORY_PATH)
        last_epoch = history_df['epoch'].max()
        print(f"Epoch terakhir: {last_epoch}")
        initial_epoch = last_epoch + 1
        print(f"Resuming training from epoch {initial_epoch}")
        model = tf.keras.models.load_model(EPOCH_MODEL_PATH)

    else:
        print("No checkpoint found. Building new model.")
        model = build_model(MODEL_NAME, len(np.unique(y_train)), BASE_MODEL_TRAINABLE)

model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()



In [ ]:
# Check if CSV log exists
if os.path.exists(HISTORY_PATH):
    import pandas as pd
    try: # Try to read the CSV file, handling potential errors
        log_data = pd.read_csv(HISTORY_PATH)
        if not log_data.empty:
            initial_epoch = log_data['epoch'].max() + 1
            print(f"Resuming training from epoch {initial_epoch}")
    except pd.errors.EmptyDataError: # If the file is empty, skip loading and proceed
        print("CSV log file is empty. Starting training from epoch 0.")

# Callbacks
callbacks_list = [
    callbacks.ModelCheckpoint(
        filepath=BEST_MODEL_PATH,
        save_best_only=True,
        save_weights_only=False,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        filepath=EPOCH_MODEL_PATH,
        save_best_only=False,
        save_weights_only=False,
        verbose=1
    ),
    callbacks.CSVLogger(HISTORY_PATH, append=True),
]

# Training
print(f"Training for {EXPERIMENT_NAME}")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    initial_epoch=initial_epoch,
    callbacks=callbacks_list
)

In [ ]:
def plot_history_from_csv(csv_path, plot_path):
    # Load training history from CSV
    history_df = pd.read_csv(csv_path)

    # Plot training and validation accuracy
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history_df['accuracy'], label='Train Accuracy')
    plt.plot(history_df['val_accuracy'], label='Validation Accuracy')
    plt.legend()
    plt.title(f"Accuracy {EXPERIMENT_NAME}")
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')

    # Plot training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(history_df['loss'], label='Train Loss')
    plt.plot(history_df['val_loss'], label='Validation Loss')
    plt.legend()
    plt.title(f"Loss {EXPERIMENT_NAME}")
    plt.xlabel('Epoch')
    plt.ylabel('Loss')

    # Save the plot
    plt.savefig(plot_path)
    plt.show()

# Call the function using CSV_LOG_PATH and PLOT_PATH
plot_history_from_csv(HISTORY_PATH, PLOT_PATH)


In [ ]:
def evaluate_model(model, test_ds):
    all_labels, all_preds = [], []
    inference_times = []  # List to store inference times

    for x, y in test_ds:
        start_time = time.time()  # Record start time
        preds = model.predict(x, verbose=1)
        end_time = time.time()  # Record end time
        inference_times.append(end_time - start_time)  # Calculate and store inference time

        all_preds.extend(np.argmax(preds, axis=1))
        all_labels.extend(y.numpy())

    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    avg_inference_time = np.mean(inference_times)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Average Inference Time: {avg_inference_time:.4f} seconds")

    cm = confusion_matrix(all_labels, all_preds)
    print("Confusion Matrix:")
    print(cm)

evaluate_model(model, test_ds)